### 직무별-테마별 요약

In [ ]:
# import libraries
import pandas as pd
import pickle
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('punkt')

# !pip install transformers[torch]==4.27.4
# !pip install protobuf==3.20

In [ ]:
# make a folder if it's not exist yet
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

##### T5
* https://github.com/AIRC-KETI/ke-t5
* https://github.com/AIRC-KETI/ke-t5#pretrained-models
* https://github.com/AIRC-KETI/ke-t5-downstreams
* https://huggingface.co/docs/transformers/model_doc/t5

In [35]:
# t5 korean pretrained model lists

# lcw99/t5-base-korean-text-summary
# model_dir = "lcw99/t5-base-korean-text-summary"
# tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# lcw99/t5-large-korean-text-summary
# https://huggingface.co/lcw99/t5-large-korean-text-summary
# model_dir = "lcw99/t5-large-korean-text-summary"
# tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# KETI-AIR/ke-t5-base-ko
# model_name = "KETI-AIR/ke-t5-base-ko"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

# KETI-AIR/ke-t5-small-ko
# tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-small-ko")
# model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR/ke-t5-small-ko")

In [47]:
# review summarization using T5 pretrained model

# make a folder to save data
folder_path = '.\\result'
makedirs(folder_path)

data_path = '..\\reviewdataset\\review_posneg_theme_byjob\\'
file_list = os.listdir(data_path)
file_list_name = [file for file in file_list if file.endswith('.pkl')]

for file in file_list_name:
	filename = file.split('.pkl')[0]

	# load a pickle file
	data_test = []

	with open('{}{}'.format(path, file), 'rb') as tf:
		data_test = pickle.load(tf)
	tf.close()

	# join list elements in one sentence
	sentence_all = '\n'.join(data_test)

	# model
	model_dir = "lcw99/t5-large-korean-text-summary"
	tokenizer = AutoTokenizer.from_pretrained(model_dir)
	model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

	# parameters
	max_input_length = 512

	inputs = ["summarize_summary: " + sentence_all]  # summarize

	inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
	output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=100)
	decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
	predicted = nltk.sent_tokenize(decoded_output.strip())[0]

	# save a predicted sentence as a text file
	with open('{}/{}_smr.txt'.format(folder_path,filename),'w') as f:
		f.write(predicted)
	f.close()